# Medicaid Provider Monthly Spending Trends

Interactive visualization of top 1000 Billing NPIs (2018-2024)

## Setup Instructions
1. Upload the following CSV files to this Colab session:
   - `monthly_summary_top1000.csv`
   - `top1000_npi_with_names.csv`
2. Run all cells in order
3. Use the dropdown and buttons to navigate between NPIs

In [ ]:
# Install required packages (if needed)
# !pip install pandas matplotlib ipywidgets

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets, HBox, VBox, Output
from IPython.display import display, clear_output
from google.colab import files

print("Libraries loaded successfully!")

In [ ]:
# Upload your CSV files
print("Please upload your CSV files:")
print("1. monthly_summary_top1000.csv")
print("2. top1000_npi_with_names.csv")
print()
uploaded = files.upload()

In [ ]:
# Load data
print("Loading data...")
df = pd.read_csv('monthly_summary_top1000.csv')
npi_names_df = pd.read_csv('top1000_npi_with_names.csv')

# Parse month into year and month number
df['year'] = df['month'].str[:4].astype(int)
df['month_num'] = df['month'].str[5:7].astype(int)

# Create NPI names dictionary
npi_names = dict(zip(npi_names_df['billing_npi'], npi_names_df['name']))

# Get list of unique billing NPIs sorted by total paid
npi_totals = df.groupby('billing_npi')['total_paid'].sum().sort_values(ascending=False)
npi_list = npi_totals.index.tolist()

print(f"Loaded {len(npi_list)} NPIs")
print(f"Data rows: {len(df):,}")

In [ ]:
# Colors and markers for each year
year_colors = {
    2018: '#1f77b4',  # blue
    2019: '#ff7f0e',  # orange
    2020: '#2ca02c',  # green
    2021: '#d62728',  # red
    2022: '#9467bd',  # purple
    2023: '#8c564b',  # brown
    2024: '#e377c2',  # pink
}

year_markers = {
    2018: 'o',   # circle
    2019: 's',   # square
    2020: '^',   # triangle up
    2021: 'D',   # diamond
    2022: 'v',   # triangle down
    2023: 'p',   # pentagon
    2024: '*',   # star
}

In [ ]:
def plot_npi(rank):
    """Plot monthly trends for a specific NPI by rank."""
    idx = rank - 1
    npi = npi_list[idx]
    npi_data = df[df['billing_npi'] == npi]
    total_paid = npi_totals[npi]
    npi_name = npi_names.get(npi, "Unknown")
    
    fig, ax = plt.subplots(figsize=(12, 7))
    
    # Plot each year as a separate line
    for year in sorted(year_colors.keys()):
        year_data = npi_data[npi_data['year'] == year].sort_values('month_num')
        if not year_data.empty:
            ax.plot(
                year_data['month_num'],
                year_data['total_paid'] / 1e6,
                marker=year_markers[year],
                color=year_colors[year],
                label=str(year),
                linewidth=2,
                markersize=8
            )
    
    ax.set_xlabel('Month', fontsize=12)
    ax.set_ylabel('Total Paid (Millions $)', fontsize=12)
    ax.set_title(
        f'{npi_name}\n'
        f'NPI: {npi} | Rank: {rank}/{len(npi_list)} | '
        f'Total Paid (2018-2024): ${total_paid:,.0f}',
        fontsize=11
    )
    ax.set_xticks(range(1, 13))
    ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                        'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    ax.legend(title='Year', loc='upper right')
    ax.grid(True, alpha=0.3)
    ax.set_ylim(bottom=0)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Interactive widget
print("Use the slider to navigate between NPIs (Rank 1 = highest spending)")
print()

interact(
    plot_npi,
    rank=widgets.IntSlider(
        value=1,
        min=1,
        max=len(npi_list),
        step=1,
        description='NPI Rank:',
        continuous_update=False,
        style={'description_width': 'initial'}
    )
);

## Alternative: Search by NPI Number

Run the cell below and enter an NPI number to find and plot it.

In [ ]:
# Search for specific NPI
search_npi = input("Enter NPI number to search: ")

try:
    search_npi_int = int(search_npi)
    if search_npi_int in npi_list:
        found_rank = npi_list.index(search_npi_int) + 1
        print(f"Found! NPI {search_npi_int} is ranked #{found_rank}")
        plot_npi(found_rank)
    else:
        print(f"NPI {search_npi_int} not found in top 1000")
except ValueError:
    print("Invalid NPI format. Please enter a number.")

## View Top 10 NPIs Summary

In [ ]:
# Display top 10 NPIs
top10_data = []
for i, npi in enumerate(npi_list[:10], 1):
    top10_data.append({
        'Rank': i,
        'NPI': npi,
        'Name': npi_names.get(npi, 'Unknown'),
        'Total Paid': f"${npi_totals[npi]:,.0f}"
    })

top10_df = pd.DataFrame(top10_data)
display(top10_df)